In [1]:
# Majority vote with 3 classifiers
# Paper based on this approach
#  doi=10.1109/TNSE.2021.3099371

In [2]:
import numpy as np
import os
import glob
import pandas as pd
from sklearn.metrics import roc_curve
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

import scikitplot as skplt
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
%matplotlib inline 
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import accuracy_score

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd



/home/manero/DL/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2023-02-17 21:42:22.095123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 21:42:22.095145: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

name = "./Files_Majority_vote/128_features/filter"
#name = "e:/$Notebooks/Original_files_128_features/filter"
#name = "G:/Darshana/Scada/Cistel/Marzia/Practical/Feature Selection/Multi Files/output files/Multifilter"
i = 0
j = 1

for i in range (1,16):
    file_name = name+str(i)+".csv"
    print(file_name)
    df_file = pd.read_csv(file_name)
    enc = LabelEncoder()
    df_file["marker"] = enc.fit_transform(df_file["marker"])
    df1_file = df_file.replace([np.inf, -np.inf], np.nan)
    m1 = df1_file.dropna()
    X= m1.iloc[:,0:14]
    y1= m1.iloc[:,15]
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y1)
    #y1 = to_categorical(y)
    sc = StandardScaler()
    X = sc.fit_transform(X)

    clf1 = GradientBoostingClassifier(n_estimators= 5, max_leaf_nodes= None, max_depth=12, 
                                  random_state= 3, min_samples_split= 12, learning_rate=0.1)
    clf2 = RandomForestClassifier(n_estimators=10,bootstrap=True, class_weight=None, 
                                  criterion='gini', max_depth=12, max_features='auto', max_leaf_nodes=None, 
                                  min_impurity_decrease=0.0, min_samples_leaf=1, 
                                  min_samples_split=2, min_weight_fraction_leaf=0.0, n_jobs=None, 
                                  oob_score=False, random_state=3, verbose=0, warm_start=False) 
    clf3 = XGBClassifier(n_estimators=10, max_leaf_nodes= None, max_depth=12, random_state= 3, 
                                  min_samples_split= 1)
    eclf = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2), ('xgb', clf3)],voting='hard')
    
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []
    fig, ax = plt.subplots()
    
    print ('n_estimators = 100 \n')
    print('10-fold cross validation for file : ', i)
    df_results = pd.DataFrame() 
    for clf, label in zip([clf1, clf2, clf3, eclf], ['Gradient Boosting', 'Random Forest', 'XGBClassifier', 'Ensemble Majority']):
        seed = 123
        kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
        cvscores = []
        for train, test in kfold.split(X,y):
            history = clf.fit(X[train],y1[train])
            scoresTrain = clf.score(X[train], y1[train]) 
            scores = clf.score(X[test], y1[test])
            print(clf)
#            visual_roc = skplt.metrics.plot_roc(label, X[test], y1[test])
#
#            interp_tpr = interp(mean_fpr, visual_roc.fpr, visual_roc.tpr)
#            interp_tpr[0] = 0.0
#            tprs.append(interp_tpr)
#            aucs.append(viz.roc_auc)
            
            
            print("kfold ",i," score: %.2f%%" % ( scores*100))
            print("kfold ",i," score: %.2f%%" % (scoresTrain*100)) 
            cvscores.append(scores * 100)
        df_results[label] = cvscores
        print("Average score over k-fold %.2f%% (+/-%.2f%%) [%s]" % (np.mean(cvscores), np.std(cvscores), label))


./Files_Majority_vote/128_features/filter1.csv
n_estimators = 100 

10-fold cross validation for file :  1
GradientBoostingClassifier(max_depth=12, min_samples_split=12, n_estimators=5,
                           random_state=3)
kfold  1  score: 83.10%
kfold  1  score: 85.59%
GradientBoostingClassifier(max_depth=12, min_samples_split=12, n_estimators=5,
                           random_state=3)
kfold  1  score: 86.32%
kfold  1  score: 88.48%
GradientBoostingClassifier(max_depth=12, min_samples_split=12, n_estimators=5,
                           random_state=3)
kfold  1  score: 83.10%
kfold  1  score: 85.68%
GradientBoostingClassifier(max_depth=12, min_samples_split=12, n_estimators=5,
                           random_state=3)
kfold  1  score: 88.93%
kfold  1  score: 88.39%
GradientBoostingClassifier(max_depth=12, min_samples_split=12, n_estimators=5,
                           random_state=3)
kfold  1  score: 88.73%
kfold  1  score: 91.54%
GradientBoostingClassifier(max_depth=12, mi

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=12, max_leaf_nodes=None,
              min_child_weight=1, min_samples_split=1, missing=nan,
              monotone_constraints='()', n_estimators=10, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=3,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
kfold  1  score: 93.36%
kfold  1  score: 99.13%
[21:46:37] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some paramet

[21:46:38] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:46:38] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

[21:46:40] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:46:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

[21:46:41] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:46:41] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

...

[21:48:46] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:48:46] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

[21:48:48] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:48:48] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

[21:48:49] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:48:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=12, max_leaf_nodes=None,
              min_child_weight=1, min_samples_split=1, missing=nan,
              monotone_constraints='()', n_estimators=10, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=3,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
kfold  15  score: 93.94%
kfold  15  score: 99.37%
[21:48:53] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some param

[21:48:54] WARNING: ../src/learner.cc:576: 
Parameters: { "min_samples_split" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:48:54] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(max_depth=12,
                                                         min_samples_split=12,
                                                         n_estimators=5,
                                                         random_state=3)),
                             ('rf',
          

In [ ]:

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
ax.legend(loc="lower right")
plt.show()

In [ ]:
est = [('gb', clf1), ('rf', clf2), ('xgb', clf3)]

In [ ]:
from matplotlib import pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 0)

vot_hard = VotingClassifier(estimators = est, voting ='soft') 
vot_hard.fit(X_train, y_train) 
y_pred = vot_hard.predict(X_test) 
y_proba = vot_hard.predict_proba(X_test)
print(y_proba[:,1])
print(y_proba.shape, y_pred.shape)
score = accuracy_score(y_test, y_pred) 
print("Hard Voting Score ", score) 

ns_fpr, ns_tpr, _ = roc_curve(y_test, y_proba[:,1])
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='Soft Vote')
plt.show()

In [ ]:
df_results

In [ ]:
df_results.mean()

In [ ]:
#EOF